
# ECE795 Advanced Big Data Analytics Homework 2 (Due 2/17 before lecture)

Set up Pyspark Environment.

Tips for Colab:

1. You will be disconnected if you are idle for more than 90 minutes and will be mandatorily disconnected after 12 hour connection. 

2. Once you got disconnected, you need to execute the codes from the beginning to setup the environment again.

3. For the purpose of homework, it should be sufficient since each problem should not take more than 5 minutes to generate the results.

4. To facilitate the use of Colab, you can use "MainMenu - Runtime - Run all” to run all the cells in the notebook. So you do not have to click each cell to setup the environment.

In [ ]:
!pip install pyspark
!pip install -q findspark

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 58.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=c0ac02968cf15c3b4d78229b4619dfb58d4ce1e2f2fd3bf031a8750039715872
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


Congrats! Your Colab is ready to run Pyspark.

# Read input text file to RDD 

Download the input data from [here](https://raw.githubusercontent.com/umddm/ECE795_Homeworks_Spring2022/homework_2/Gutenberg.txt).

In [ ]:
!wget https://raw.githubusercontent.com/umddm/ECE795_Homeworks_Spring2022/homework_2/Gutenberg.txt

--2022-02-16 05:31:11--  https://raw.githubusercontent.com/umddm/ECE795_Homeworks_Spring2022/homework_2/Gutenberg.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3536418 (3.4M) [text/plain]
Saving to: ‘Gutenberg.txt’

Gutenberg.txt       100%[===================>]   3.37M  --.-KB/s    in 0.05s   

2022-02-16 05:31:12 (71.7 MB/s) - ‘Gutenberg.txt’ saved [3536418/3536418]



Now that we have input data, we can start to do the homework. 

## Question 1 (4 points): The following RDD, 'sentences', contains all the sentences in the text file. Please use 'flatMap' function to create another RDD called 'words' that contains all the words in the text file, then count the number of distinct words in this RDD.


In [ ]:
# The following code is from the previous homework
from pyspark import SparkConf, SparkContext
def removeNonAlpabet(s):
    return ''.join([i.lower() for i in s if i.isalpha() or i==' ']).lstrip().rstrip()

sc = SparkContext.getOrCreate()
sentences = sc.textFile('Gutenberg.txt').map(removeNonAlpabet).filter(lambda x: x!='')


#Question_1:
#Fill out here

words = sentences.flatMap(lambda line: line.split())
distinct_words = words.distinct()
distinct_words.count()

104116

## Question 2 (5 points): For the 'words' RDD in Question 1, please find the its top 10 most frequent words.

In [ ]:
#Question_2:
#Fill out here

c_words = words.map(lambda word: (word,1))
c_words = c_words.reduceByKey(lambda v1, v2: v1 + v2)
c_words = c_words.map(lambda x: (x[1], x[0]))
c_words = c_words.groupByKey().mapValues(list)
c_words.top(10)

[(6908, ['the']),
 (6853, ['subst']),
 (6820, ['of']),
 (6800, ['to']),
 (5828, ['in']),
 (5722, ['van']),
 (4875, ['de']),
 (4469, ['a']),
 (4107, ['een']),
 (3711, ['het'])]

## Question 3 (5 points): The following code will create another RDD called 'stop_words' that contains all the stop words in English. For the 'words' RDD in Question 1, please count its number of words that do not appear in 'stop_words'.


In [ ]:
#Question_3
# The following code will create another RDD called 'stop_words' that contains all the 'stop words' in English.
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = sc.parallelize(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#Question_3
#Fill out here

words.subtract(stop_words).count() # 373182  

# For the 'words' RDD in Question 1 ? 
# If the 'words' means distinct words, the solution should be 103978.       103978/104116
# If the 'words' means words (not distinct), the solution should be 373182.    373182/446265
# distinct_words = words.distinct()
# distinct_words.subtract(stop_words).count()  # 103978, 138/179


373182

## Question 4 (16 points, 2 points for each function): Assume an RDD is created from a list of numbers, please determine whether it's appropriate to use each of the the following functions in a 'reduce' operation or not.



In [ ]:
#Question_4
'''
Requirements:
1. commutative: f(x, y) = f(y, x)
2. associative: f(f(x, y), z) = f(x, f(y, z))
'''

#Function A
'''
Solution: No. For example, f[(1, 2), 3] = 1, f[1, (2, 3)] = 0. f[(1, 2), 3] != f[1, (2, 3)]
'''
def functionA(x, y):
  if x != y:
    return 1
  else:
    return 0

#Function B
'''
Solution: No. For example, f(1, 2) = 1, f(2, 1) = 0. f(1, 2) != f(2, 1)
'''
def functionB(x, y):
  if x < y :
    return x
  elif y > x:  
    return y
  else:
    return 0

#Function C
'''
Solution: Yes. f(x, y) = f(y, x), and f(f(x, y), z) = f(x, f(y, z))
'''
def functionC(x, y):
  return x - 10 + y

#Function D
'''
Solution: Yes. f(x, y) = f(y, x), and f(f(x, y), z) = f(x, f(y, z))
'''
def functionD(x, y):
  return x * y + x + y

#Function E
'''
Solution: No. For example, f[(1, 2), 3] = 12*12, f[1, (2, 3)] = 26*26. f[(1, 2), 3] != f[1, (2, 3)]
'''
def functionE(x, y):
  return (x + y) ** 2

#Function F
'''
Solution: No. For example, f[(1, 2), 3] = 17/4, f[1, (2, 3)] = 9/2. f[(1, 2), 3] != f[1, (2, 3)]
'''
def functionF(x, y):
  return x / 2 + y

#Function G
'''
Solution: No. For example, f(1, 2) = 9, f(2, 1) = 5. f(x, y) != f(y, x)
'''
def functionG(x, y):
  return x ** 2 + y ** 3

#Function H
'''
Solution: No. For example, f(1, 2) = -1/2, f(2, 1) = 1/2. f(x, y) != f(y, x)
'''
def functionH(x, y):
  return (x - y) / 2